In [2]:
import  cv2
import  os
import h5py
from tqdm import tqdm
from PIL import Image
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from glob import glob
import shutil

In [4]:
size=500

In [3]:
X_train = np.zeros((2022,size,size,3),dtype=np.uint8) 
y_train = np.zeros((2022,),dtype=np.uint8)

j = 0
for img_path in tqdm(glob("{0}/*.jpg".format('train_changed/1'))):

    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(size,size),interpolation=cv2.INTER_AREA)

    #原图直接resize
    X_train[j,:,:,:] = img
    y_train[j] = 1
    j += 1

for img_path in tqdm(glob("{0}/*.jpg".format('train_changed/0'))):

    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(size,size),interpolation=cv2.INTER_AREA)

    #原图直接resize
    X_train[j,:,:,:] = img
    y_train[j] = 0
    j += 1

  1%|█                                                                                 | 9/706 [00:01<02:30,  4.64it/s]


KeyboardInterrupt: 

In [4]:
X_test = np.zeros((647,size,size,3),dtype=np.uint8) 
j = 0

for img_path in tqdm(glob("{0}/*.jpg".format('xuelang_round1_test_b'))):

    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(size,size),interpolation=cv2.INTER_AREA)

    #原图直接resize
    X_test[j,:,:,:] = img
    j += 1

100%|████████████████████████████████████████████████████████████████████████████████| 647/647 [01:02<00:00, 10.43it/s]


In [ ]:
import h5py

with h5py.File('biliao_Data_{0}'.format(size)) as h:
    h.create_dataset("train",data = X_train)
    h.create_dataset("test",data = X_test)
    h.create_dataset('label',data = y_train)

In [3]:
with h5py.File('biliao_Data_{0}'.format(size)) as h:
    X_train = np.array(h['train'])
    X_test = np.array(h['test'])
    y_train = np.array(h['label'])

NameError: name 'size' is not defined

查看下有没有空值：

In [39]:
X_test.shape

(662, 500, 500, 3)

In [40]:
for i in range(662):
    if X_test[i].sum() == 0:
        print(i)

In [41]:
y_train.sum()

706

分割训练集和验证集

In [8]:
from sklearn.utils import shuffle
np.random.seed(2018)
X_train,y_train = shuffle(X_train,y_train)

In [9]:
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val = train_test_split(X_train, y_train, test_size=0.1,stratify=y_train,random_state=2018)

In [10]:
import tensorflow as tf
def as_keras_metric(method):
    import functools
    from keras import backend as K
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper
auc_roc = as_keras_metric(tf.metrics.auc)

In [11]:
from keras.layers import *
from keras.models import *
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.optimizers import *
from keras.applications import *


def MODEL(MODEL,h,w,out_dims,func=None,weights=None,include_top=False):
    inputs = Input((h,w,3)) #实例化一个tensor
    x = inputs
    x = Lambda(func)(x)
#     x = Conv2D(16,(3,3),padding='same',activation='relu')(x)
#     x = Conv2D(32,(3,3),padding='same',activation='relu')(x)
#     x = Conv2D(3,(3,3),padding='same',activation='relu')(x)
#     x = MaxPooling2D((3,3),(2,2),padding='same')(x)
    base_model = MODEL(weights=weights, include_top=include_top) #要提取特征，最后的一层全连接层要去掉
    x = base_model(x)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(out_dims, activation='sigmoid')(x)
    model = Model(inputs, x)
    return model

In [12]:
from tensorflow.python.keras.applications.inception_v3 import preprocess_input

model = MODEL(InceptionV3,size,size,out_dims=1,func=preprocess_input,weights='imagenet')

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 500, 500, 3)       0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 500, 500, 3)       0         
_________________________________________________________________
inception_v3 (Model)         multiple                  21802784  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2049      
Total params: 21,804,833
Trainable params: 21,770,401
Non-trainable params: 34,432
___________________________________________________________

In [14]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(width_shift_range = 0.1, 
                                 height_shift_range = 0.1, 
                                 horizontal_flip = True,
                                 vertical_flip = True,
                                 zoom_range = 0.1)
val_datagen = ImageDataGenerator()

batch_size = 8

train_generator = train_datagen.flow(X_train,y_train,batch_size=batch_size,shuffle=False)
val_generator = val_datagen.flow(X_val,y_val,batch_size=batch_size,shuffle=False)

In [ ]:
checkpointer = ModelCheckpoint(filepath='InceptionV3_500_training.hdf5', monitor='val_auc',mode='max',
                               verbose=1, save_best_only=True)
reduce = ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=5,verbose=1)
adam = Adam(0.0001)
model.compile(optimizer = adam,
              loss='binary_crossentropy',
              metrics=[auc_roc])
# model.load_weights('InceptionV3_500_0.98520.hdf5')

epochs = 50

#model = model.load_weights('buliao.hdf5')
history = model.fit_generator(train_generator,
          validation_data = val_generator,
          epochs=epochs,
          callbacks=[checkpointer,reduce],
          verbose=1)

Epoch 1/50
 73/228 [========>.....................] - ETA: 1:10:30 - loss: 0.5784 - auc: 0.0000e+ - ETA: 35:35 - loss: 0.5309 - auc: 0.4000     - ETA: 23:56 - loss: 0.5829 - auc: 0.53 - ETA: 18:07 - loss: 0.5728 - auc: 0.58 - ETA: 14:37 - loss: 0.5816 - auc: 0.61 - ETA: 12:17 - loss: 0.6271 - auc: 0.62 - ETA: 10:37 - loss: 0.6171 - auc: 0.62 - ETA: 9:22 - loss: 0.6119 - auc: 0.6200 - ETA: 8:24 - loss: 0.5961 - auc: 0.620 - ETA: 7:37 - loss: 0.5909 - auc: 0.623 - ETA: 6:59 - loss: 0.6090 - auc: 0.625 - ETA: 6:26 - loss: 0.6059 - auc: 0.626 - ETA: 5:59 - loss: 0.6073 - auc: 0.627 - ETA: 5:36 - loss: 0.6187 - auc: 0.628 - ETA: 5:16 - loss: 0.6061 - auc: 0.629 - ETA: 4:58 - loss: 0.6027 - auc: 0.630 - ETA: 4:42 - loss: 0.5907 - auc: 0.631 - ETA: 4:28 - loss: 0.5816 - auc: 0.633 - ETA: 4:16 - loss: 0.5681 - auc: 0.635 - ETA: 4:05 - loss: 0.5631 - auc: 0.638 - ETA: 3:54 - loss: 0.5575 - auc: 0.642 - ETA: 3:45 - loss: 0.5665 - auc: 0.645 - ETA: 3:36 - loss: 0.5644 - auc: 0.647 - ETA: 3:29 - l

In [17]:
history = model.fit_generator(train_generator,
          validation_data = val_generator,
          epochs=50,
          callbacks=[checkpointer],
          verbose=1)

Epoch 1/50
210/228 [==========================>...] - ETA: 2:29 - loss: 0.0307 - auc: 0.983 - ETA: 2:00 - loss: 0.0220 - auc: 0.983 - ETA: 1:51 - loss: 0.0232 - auc: 0.983 - ETA: 1:47 - loss: 0.0289 - auc: 0.983 - ETA: 1:44 - loss: 0.0239 - auc: 0.983 - ETA: 1:41 - loss: 0.0213 - auc: 0.983 - ETA: 1:39 - loss: 0.0278 - auc: 0.983 - ETA: 1:38 - loss: 0.0245 - auc: 0.983 - ETA: 1:37 - loss: 0.0229 - auc: 0.983 - ETA: 1:36 - loss: 0.0210 - auc: 0.983 - ETA: 1:35 - loss: 0.0204 - auc: 0.983 - ETA: 1:35 - loss: 0.0189 - auc: 0.983 - ETA: 1:33 - loss: 0.0178 - auc: 0.983 - ETA: 1:33 - loss: 0.0169 - auc: 0.983 - ETA: 1:32 - loss: 0.0169 - auc: 0.983 - ETA: 1:31 - loss: 0.0162 - auc: 0.983 - ETA: 1:31 - loss: 0.0157 - auc: 0.983 - ETA: 1:30 - loss: 0.0159 - auc: 0.983 - ETA: 1:29 - loss: 0.0289 - auc: 0.983 - ETA: 1:29 - loss: 0.0286 - auc: 0.983 - ETA: 1:28 - loss: 0.0276 - auc: 0.983 - ETA: 1:28 - loss: 0.0268 - auc: 0.983 - ETA: 1:27 - loss: 0.0276 - auc: 0.983 - ETA: 1:27 - loss: 0.0302 -

212/228 [==========================>...] - ETA: 1:01 - loss: 0.1521 - auc: 0.983 - ETA: 59s - loss: 0.0843 - auc: 0.983 - ETA: 58s - loss: 0.1577 - auc: 0.98 - ETA: 58s - loss: 0.1246 - auc: 0.98 - ETA: 57s - loss: 0.1163 - auc: 0.98 - ETA: 57s - loss: 0.1339 - auc: 0.98 - ETA: 57s - loss: 0.1182 - auc: 0.98 - ETA: 56s - loss: 0.1547 - auc: 0.98 - ETA: 56s - loss: 0.1389 - auc: 0.98 - ETA: 56s - loss: 0.1296 - auc: 0.98 - ETA: 55s - loss: 0.1262 - auc: 0.98 - ETA: 55s - loss: 0.1326 - auc: 0.98 - ETA: 56s - loss: 0.1259 - auc: 0.98 - ETA: 58s - loss: 0.1181 - auc: 0.98 - ETA: 1:00 - loss: 0.1148 - auc: 0.983 - ETA: 1:01 - loss: 0.1081 - auc: 0.983 - ETA: 1:03 - loss: 0.1047 - auc: 0.983 - ETA: 1:04 - loss: 0.1026 - auc: 0.983 - ETA: 1:05 - loss: 0.0974 - auc: 0.983 - ETA: 1:05 - loss: 0.0951 - auc: 0.983 - ETA: 1:06 - loss: 0.0948 - auc: 0.983 - ETA: 1:07 - loss: 0.0927 - auc: 0.983 - ETA: 1:07 - loss: 0.0910 - auc: 0.983 - ETA: 1:08 - loss: 0.0880 - auc: 0.983 - ETA: 1:08 - loss: 0.08

228/228 [==============================] - ETA: 6s - loss: 0.0512 - auc: 0.984 - ETA: 5s - loss: 0.0511 - auc: 0.984 - ETA: 5s - loss: 0.0509 - auc: 0.984 - ETA: 4s - loss: 0.0508 - auc: 0.984 - ETA: 4s - loss: 0.0511 - auc: 0.984 - ETA: 4s - loss: 0.0538 - auc: 0.984 - ETA: 3s - loss: 0.0537 - auc: 0.984 - ETA: 3s - loss: 0.0535 - auc: 0.984 - ETA: 2s - loss: 0.0533 - auc: 0.984 - ETA: 2s - loss: 0.0535 - auc: 0.984 - ETA: 2s - loss: 0.0537 - auc: 0.984 - ETA: 1s - loss: 0.0535 - auc: 0.984 - ETA: 1s - loss: 0.0533 - auc: 0.984 - ETA: 0s - loss: 0.0532 - auc: 0.984 - ETA: 0s - loss: 0.0531 - auc: 0.984 - 96s 419ms/step - loss: 0.0530 - auc: 0.9840 - val_loss: 0.6507 - val_auc: 0.9840

Epoch 00002: val_auc improved from 0.98394 to 0.98398, saving model to InceptionV3_500_training.hdf5
Epoch 3/50
212/228 [==========================>...] - ETA: 59s - loss: 0.0901 - auc: 0.98 - ETA: 58s - loss: 0.0598 - auc: 0.98 - ETA: 58s - loss: 0.0417 - auc: 0.98 - ETA: 57s - loss: 0.0345 - auc: 0.98 

212/228 [==========================>...] - ETA: 1:01 - loss: 0.0092 - auc: 0.984 - ETA: 59s - loss: 0.0219 - auc: 0.984 - ETA: 58s - loss: 0.0274 - auc: 0.98 - ETA: 58s - loss: 0.0264 - auc: 0.98 - ETA: 57s - loss: 0.0235 - auc: 0.98 - ETA: 57s - loss: 0.0208 - auc: 0.98 - ETA: 57s - loss: 0.0199 - auc: 0.98 - ETA: 56s - loss: 0.0202 - auc: 0.98 - ETA: 56s - loss: 0.0218 - auc: 0.98 - ETA: 56s - loss: 0.0199 - auc: 0.98 - ETA: 55s - loss: 0.0215 - auc: 0.98 - ETA: 55s - loss: 0.0218 - auc: 0.98 - ETA: 56s - loss: 0.0208 - auc: 0.98 - ETA: 58s - loss: 0.0215 - auc: 0.98 - ETA: 1:00 - loss: 0.0341 - auc: 0.984 - ETA: 1:02 - loss: 0.0346 - auc: 0.984 - ETA: 1:03 - loss: 0.0344 - auc: 0.984 - ETA: 1:04 - loss: 0.0341 - auc: 0.984 - ETA: 1:05 - loss: 0.0406 - auc: 0.984 - ETA: 1:06 - loss: 0.0394 - auc: 0.984 - ETA: 1:07 - loss: 0.0409 - auc: 0.984 - ETA: 1:07 - loss: 0.0393 - auc: 0.984 - ETA: 1:08 - loss: 0.0404 - auc: 0.984 - ETA: 1:08 - loss: 0.0406 - auc: 0.984 - ETA: 1:09 - loss: 0.04

228/228 [==============================] - ETA: 6s - loss: 0.0430 - auc: 0.984 - ETA: 5s - loss: 0.0429 - auc: 0.984 - ETA: 5s - loss: 0.0433 - auc: 0.984 - ETA: 4s - loss: 0.0441 - auc: 0.984 - ETA: 4s - loss: 0.0441 - auc: 0.984 - ETA: 4s - loss: 0.0443 - auc: 0.984 - ETA: 3s - loss: 0.0441 - auc: 0.984 - ETA: 3s - loss: 0.0441 - auc: 0.984 - ETA: 2s - loss: 0.0440 - auc: 0.984 - ETA: 2s - loss: 0.0442 - auc: 0.984 - ETA: 2s - loss: 0.0442 - auc: 0.984 - ETA: 1s - loss: 0.0444 - auc: 0.984 - ETA: 1s - loss: 0.0453 - auc: 0.984 - ETA: 0s - loss: 0.0452 - auc: 0.984 - ETA: 0s - loss: 0.0456 - auc: 0.984 - 96s 421ms/step - loss: 0.0455 - auc: 0.9840 - val_loss: 0.6531 - val_auc: 0.9841

Epoch 00004: val_auc improved from 0.98402 to 0.98406, saving model to InceptionV3_500_training.hdf5
Epoch 5/50
212/228 [==========================>...] - ETA: 1:01 - loss: 0.0311 - auc: 0.984 - ETA: 59s - loss: 0.0517 - auc: 0.984 - ETA: 58s - loss: 0.0382 - auc: 0.98 - ETA: 58s - loss: 0.0406 - auc: 0.

...

228/228 [==============================] - ETA: 6s - loss: 0.0509 - auc: 0.985 - ETA: 5s - loss: 0.0507 - auc: 0.985 - ETA: 5s - loss: 0.0505 - auc: 0.985 - ETA: 4s - loss: 0.0507 - auc: 0.985 - ETA: 4s - loss: 0.0517 - auc: 0.985 - ETA: 4s - loss: 0.0515 - auc: 0.985 - ETA: 3s - loss: 0.0514 - auc: 0.985 - ETA: 3s - loss: 0.0512 - auc: 0.985 - ETA: 2s - loss: 0.0512 - auc: 0.985 - ETA: 2s - loss: 0.0511 - auc: 0.985 - ETA: 2s - loss: 0.0509 - auc: 0.985 - ETA: 1s - loss: 0.0509 - auc: 0.985 - ETA: 1s - loss: 0.0508 - auc: 0.985 - ETA: 0s - loss: 0.0506 - auc: 0.985 - ETA: 0s - loss: 0.0507 - auc: 0.985 - 95s 417ms/step - loss: 0.0507 - auc: 0.9851 - val_loss: 0.6677 - val_auc: 0.9851

Epoch 00046: val_auc improved from 0.98511 to 0.98513, saving model to InceptionV3_500_training.hdf5
Epoch 47/50
212/228 [==========================>...] - ETA: 1:01 - loss: 0.0087 - auc: 0.985 - ETA: 59s - loss: 0.0168 - auc: 0.985 - ETA: 58s - loss: 0.0571 - auc: 0.98 - ETA: 58s - loss: 0.0467 - auc: 0

212/228 [==========================>...] - ETA: 1:00 - loss: 0.0092 - auc: 0.985 - ETA: 58s - loss: 0.0861 - auc: 0.985 - ETA: 58s - loss: 0.0597 - auc: 0.98 - ETA: 57s - loss: 0.0564 - auc: 0.98 - ETA: 57s - loss: 0.0562 - auc: 0.98 - ETA: 57s - loss: 0.0485 - auc: 0.98 - ETA: 57s - loss: 0.0469 - auc: 0.98 - ETA: 56s - loss: 0.0474 - auc: 0.98 - ETA: 56s - loss: 0.0502 - auc: 0.98 - ETA: 56s - loss: 0.0646 - auc: 0.98 - ETA: 55s - loss: 0.0590 - auc: 0.98 - ETA: 55s - loss: 0.0560 - auc: 0.98 - ETA: 55s - loss: 0.0523 - auc: 0.98 - ETA: 57s - loss: 0.0491 - auc: 0.98 - ETA: 59s - loss: 0.0602 - auc: 0.98 - ETA: 1:00 - loss: 0.0570 - auc: 0.985 - ETA: 1:02 - loss: 0.0547 - auc: 0.985 - ETA: 1:03 - loss: 0.0592 - auc: 0.985 - ETA: 1:04 - loss: 0.0567 - auc: 0.985 - ETA: 1:05 - loss: 0.0545 - auc: 0.985 - ETA: 1:05 - loss: 0.0545 - auc: 0.985 - ETA: 1:06 - loss: 0.0527 - auc: 0.985 - ETA: 1:07 - loss: 0.0524 - auc: 0.985 - ETA: 1:07 - loss: 0.0557 - auc: 0.985 - ETA: 1:07 - loss: 0.0554

228/228 [==============================] - ETA: 6s - loss: 0.0401 - auc: 0.985 - ETA: 5s - loss: 0.0403 - auc: 0.985 - ETA: 5s - loss: 0.0407 - auc: 0.985 - ETA: 4s - loss: 0.0405 - auc: 0.985 - ETA: 4s - loss: 0.0407 - auc: 0.985 - ETA: 4s - loss: 0.0411 - auc: 0.985 - ETA: 3s - loss: 0.0412 - auc: 0.985 - ETA: 3s - loss: 0.0413 - auc: 0.985 - ETA: 2s - loss: 0.0413 - auc: 0.985 - ETA: 2s - loss: 0.0417 - auc: 0.985 - ETA: 2s - loss: 0.0415 - auc: 0.985 - ETA: 1s - loss: 0.0416 - auc: 0.985 - ETA: 1s - loss: 0.0414 - auc: 0.985 - ETA: 0s - loss: 0.0414 - auc: 0.985 - ETA: 0s - loss: 0.0412 - auc: 0.985 - 95s 418ms/step - loss: 0.0417 - auc: 0.9852 - val_loss: 0.6497 - val_auc: 0.9852

Epoch 00048: val_auc improved from 0.98514 to 0.98517, saving model to InceptionV3_500_training.hdf5
Epoch 49/50
212/228 [==========================>...] - ETA: 1:02 - loss: 0.0485 - auc: 0.985 - ETA: 1:00 - loss: 0.0388 - auc: 0.985 - ETA: 59s - loss: 0.0323 - auc: 0.985 - ETA: 58s - loss: 0.0280 - auc:

212/228 [==========================>...] - ETA: 1:01 - loss: 0.1833 - auc: 0.985 - ETA: 59s - loss: 0.1114 - auc: 0.985 - ETA: 58s - loss: 0.0901 - auc: 0.98 - ETA: 57s - loss: 0.0708 - auc: 0.98 - ETA: 57s - loss: 0.0657 - auc: 0.98 - ETA: 57s - loss: 0.0551 - auc: 0.98 - ETA: 56s - loss: 0.0570 - auc: 0.98 - ETA: 56s - loss: 0.0506 - auc: 0.98 - ETA: 56s - loss: 0.0597 - auc: 0.98 - ETA: 56s - loss: 0.0544 - auc: 0.98 - ETA: 55s - loss: 0.0512 - auc: 0.98 - ETA: 55s - loss: 0.0727 - auc: 0.98 - ETA: 56s - loss: 0.0676 - auc: 0.98 - ETA: 59s - loss: 0.0719 - auc: 0.98 - ETA: 1:00 - loss: 0.0677 - auc: 0.985 - ETA: 1:02 - loss: 0.0646 - auc: 0.985 - ETA: 1:03 - loss: 0.0616 - auc: 0.985 - ETA: 1:04 - loss: 0.0584 - auc: 0.985 - ETA: 1:05 - loss: 0.0557 - auc: 0.985 - ETA: 1:06 - loss: 0.0552 - auc: 0.985 - ETA: 1:06 - loss: 0.0527 - auc: 0.985 - ETA: 1:07 - loss: 0.0513 - auc: 0.985 - ETA: 1:07 - loss: 0.0493 - auc: 0.985 - ETA: 1:08 - loss: 0.0473 - auc: 0.985 - ETA: 1:08 - loss: 0.04

228/228 [==============================] - ETA: 6s - loss: 0.0551 - auc: 0.985 - ETA: 5s - loss: 0.0549 - auc: 0.985 - ETA: 5s - loss: 0.0547 - auc: 0.985 - ETA: 4s - loss: 0.0546 - auc: 0.985 - ETA: 4s - loss: 0.0549 - auc: 0.985 - ETA: 4s - loss: 0.0547 - auc: 0.985 - ETA: 3s - loss: 0.0550 - auc: 0.985 - ETA: 3s - loss: 0.0548 - auc: 0.985 - ETA: 2s - loss: 0.0547 - auc: 0.985 - ETA: 2s - loss: 0.0546 - auc: 0.985 - ETA: 2s - loss: 0.0546 - auc: 0.985 - ETA: 1s - loss: 0.0544 - auc: 0.985 - ETA: 1s - loss: 0.0544 - auc: 0.985 - ETA: 0s - loss: 0.0543 - auc: 0.985 - ETA: 0s - loss: 0.0541 - auc: 0.985 - 96s 421ms/step - loss: 0.0540 - auc: 0.9852 - val_loss: 0.6555 - val_auc: 0.9852

Epoch 00050: val_auc improved from 0.98518 to 0.98520, saving model to InceptionV3_500_training.hdf5


生成全部训练集

In [17]:
X_train2 = np.concatenate((X_train,X_val),axis=0)
y_train2 = np.concatenate((y_train,y_val),axis=0)

In [ ]:
from keras.models import Model
 
base_model =  MODEL(InceptionV3,500,500,out_dims=1,func=preprocess_input,weights=None)
base_model.load_weights('buliao_2018.7.30.15:30.hdf5')

model = Model(inputs=base_model.input, outputs=base_model.layers[-3].output)

In [ ]:
X_feature = model.predict(X_train2)

In [ ]:
import h5py

with h5py.File('inceptionv3_feature_500.h5') as h:
    h.create_dataset("train",data = y_feature1)

In [ ]:
checkpointer = ModelCheckpoint(filepath='buliao_2018.7.30.17:30.hdf5', 
                               verbose=1, save_best_only=True)
reduce = ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=3,verbose=1,epsilon=1e-4)
adam = Adam(0.000025)
model.compile(optimizer = adam,
              loss='binary_crossentropy',
              metrics=[auc_roc])

model.load_weights('buliao_2018.7.30.16:08.hdf5')

epochs = 10

#model = model.load_weights('buliao.hdf5')
history = model.fit_generator(train_generator,
          validation_data = val_generator,
          epochs=epochs,
          callbacks=[checkpointer,reduce],
          verbose=1)

In [ ]:

checkpointer = ModelCheckpoint(filepath='buliao_2018.7.30.15:30.hdf5', 
                               verbose=1, save_best_only=True)

model.load_weights('buliao_2018.7.30.14:57.hdf5')
adam = Adam(0.00005)
model.compile(optimizer = adam,
              loss='binary_crossentropy',
              metrics=[auc_roc])

epochs = 5

#model = model.load_weights('buliao.hdf5')
history = model.fit_generator(train_generator,
          validation_data = val_generator,
          epochs=epochs,
          callbacks=[checkpointer],
          verbose=1)

In [ ]:
model.load_weights('buliao_2018.7.30.15:30.hdf5')
checkpointer = ModelCheckpoint(filepath='buliao_2018.7.30.16:00.hdf5', 
                               verbose=1, save_best_only=True) 
adam = Adam(0.000045)
model.compile(optimizer = adam,
              loss='binary_crossentropy',
              metrics=[auc_roc])

epochs = 10

#model = model.load_weights('buliao.hdf5')
history = model.fit_generator(train_generator,
          validation_data = val_generator,
          epochs=epochs,
          callbacks=[checkpointer],
          verbose=1)

In [34]:
import pandas as pd
submit2 = pd.read_csv('submit_mixAUC_LOSS.csv')
submit1 = pd.read_csv('submit_val_auc_0.96279_dropout_0.5.csv')

In [35]:
copy1 = submit1
copy2 = submit2

In [36]:
copy3 = pd.merge(copy1,copy2,on='filename')


In [39]:
for i in range(662):
    print(copy3.loc[i])

filename         J01_2018.06.13 13_22_11.jpg
probability_x                       0.958841
probability_y                          0.998
Name: 0, dtype: object
filename         J01_2018.06.13 13_29_45.jpg
probability_x                       0.100786
probability_y                       0.182495
Name: 1, dtype: object
filename         J01_2018.06.13 13_43_25.jpg
probability_x                         0.0005
probability_y                      0.0527841
Name: 2, dtype: object
filename         J01_2018.06.13 13_51_57.jpg
probability_x                         0.9995
probability_y                       0.919215
Name: 3, dtype: object
filename         J01_2018.06.13 13_54_49.jpg
probability_x                         0.9995
probability_y                          0.998
Name: 4, dtype: object
filename         J01_2018.06.13 14_01_56.jpg
probability_x                        0.99629
probability_y                        0.68941
Name: 5, dtype: object
filename         J01_2018.06.13 14_02_18.jpg
probabi

Name: 51, dtype: object
filename         J01_2018.06.17 09_06_17.jpg
probability_x                         0.9995
probability_y                       0.984069
Name: 52, dtype: object
filename         J01_2018.06.17 09_13_50.jpg
probability_x                         0.0005
probability_y                       0.211469
Name: 53, dtype: object
filename         J01_2018.06.17 09_15_07.jpg
probability_x                         0.9995
probability_y                          0.998
Name: 54, dtype: object
filename         J01_2018.06.17 09_21_10.jpg
probability_x                      0.0120362
probability_y                       0.125014
Name: 55, dtype: object
filename         J01_2018.06.17 09_25_19.jpg
probability_x                      0.0090238
probability_y                       0.140797
Name: 56, dtype: object
filename         J01_2018.06.17 09_27_37.jpg
probability_x                        0.35863
probability_y                       0.714854
Name: 57, dtype: object
filename         J01_2

Name: 103, dtype: object
filename         J01_2018.06.17 15_14_53.jpg
probability_x                         0.9995
probability_y                          0.998
Name: 104, dtype: object
filename         J01_2018.06.17 15_22_32.jpg
probability_x                         0.9995
probability_y                          0.998
Name: 105, dtype: object
filename         J01_2018.06.17 15_29_15.jpg
probability_x                      0.0167219
probability_y                     0.00932888
Name: 106, dtype: object
filename         J01_2018.06.17 15_33_43.jpg
probability_x                        0.99668
probability_y                          0.998
Name: 107, dtype: object
filename         J01_2018.06.17 15_51_55.jpg
probability_x                       0.183665
probability_y                       0.283299
Name: 108, dtype: object
filename         J01_2018.06.17 15_52_06.jpg
probability_x                         0.0005
probability_y                     0.00741539
Name: 109, dtype: object
filename       

Name: 155, dtype: object
filename         J01_2018.06.19 14_50_47.jpg
probability_x                         0.9995
probability_y                       0.976095
Name: 156, dtype: object
filename         J01_2018.06.19 14_57_04.jpg
probability_x                         0.9995
probability_y                       0.994546
Name: 157, dtype: object
filename         J01_2018.06.19 15_01_58.jpg
probability_x                       0.919508
probability_y                       0.229143
Name: 158, dtype: object
filename         J01_2018.06.19 15_04_09.jpg
probability_x                         0.9995
probability_y                          0.998
Name: 159, dtype: object
filename         J01_2018.06.19 15_04_20.jpg
probability_x                         0.0005
probability_y                      0.0191946
Name: 160, dtype: object
filename         J01_2018.06.19 15_10_05.jpg
probability_x                         0.9995
probability_y                          0.998
Name: 161, dtype: object
filename       

Name: 207, dtype: object
filename         J01_2018.06.22 08_48_42.jpg
probability_x                         0.9995
probability_y                          0.998
Name: 208, dtype: object
filename         J01_2018.06.22 08_48_50.jpg
probability_x                         0.0005
probability_y                          0.002
Name: 209, dtype: object
filename         J01_2018.06.22 08_51_14.jpg
probability_x                     0.00201691
probability_y                     0.00773738
Name: 210, dtype: object
filename         J01_2018.06.22 08_52_08.jpg
probability_x                         0.0005
probability_y                     0.00468986
Name: 211, dtype: object
filename         J01_2018.06.22 08_55_18.jpg
probability_x                         0.0005
probability_y                      0.0649723
Name: 212, dtype: object
filename         J01_2018.06.22 08_58_40.jpg
probability_x                         0.0005
probability_y                      0.0309212
Name: 213, dtype: object
filename       

...

Name: 415, dtype: object
filename         J01_2018.06.25 15_26_53.jpg
probability_x                     0.00885625
probability_y                      0.0184123
Name: 416, dtype: object
filename         J01_2018.06.25 15_27_30.jpg
probability_x                         0.9995
probability_y                          0.998
Name: 417, dtype: object
filename         J01_2018.06.25 15_34_17.jpg
probability_x                         0.0005
probability_y                     0.00594473
Name: 418, dtype: object
filename         J01_2018.06.25 15_41_54.jpg
probability_x                         0.0005
probability_y                      0.0135666
Name: 419, dtype: object
filename         J01_2018.06.25 15_43_08.jpg
probability_x                         0.0005
probability_y                      0.0198942
Name: 420, dtype: object
filename         J01_2018.06.26 08_43_20.jpg
probability_x                     0.00197707
probability_y                       0.126788
Name: 421, dtype: object
filename       

Name: 467, dtype: object
filename         J01_2018.06.26 13_47_34.jpg
probability_x                         0.0005
probability_y                      0.0166688
Name: 468, dtype: object
filename         J01_2018.06.26 13_47_43.jpg
probability_x                         0.0005
probability_y                          0.002
Name: 469, dtype: object
filename         J01_2018.06.26 13_48_58.jpg
probability_x                         0.0005
probability_y                          0.002
Name: 470, dtype: object
filename         J01_2018.06.26 13_58_15.jpg
probability_x                         0.9995
probability_y                          0.998
Name: 471, dtype: object
filename         J01_2018.06.26 13_59_52.jpg
probability_x                      0.0430853
probability_y                       0.023303
Name: 472, dtype: object
filename         J01_2018.06.26 14_03_16.jpg
probability_x                         0.0005
probability_y                          0.002
Name: 473, dtype: object
filename       

Name: 519, dtype: object
filename         J01_2018.06.26 15_50_37.jpg
probability_x                         0.9995
probability_y                       0.986504
Name: 520, dtype: object
filename         J01_2018.06.26 15_50_56.jpg
probability_x                    0.000706402
probability_y                       0.011638
Name: 521, dtype: object
filename         J01_2018.06.26 15_51_55.jpg
probability_x                       0.273203
probability_y                      0.0344216
Name: 522, dtype: object
filename         J01_2018.06.26 15_52_54.jpg
probability_x                      0.0371127
probability_y                      0.0287815
Name: 523, dtype: object
filename         J01_2018.06.26 15_53_24.jpg
probability_x                    0.000961206
probability_y                      0.0131485
Name: 524, dtype: object
filename         J01_2018.06.26 15_55_19.jpg
probability_x                      0.0115871
probability_y                      0.0314297
Name: 525, dtype: object
filename       

Name: 571, dtype: object
filename         J01_2018.06.27 13_30_08.jpg
probability_x                       0.227396
probability_y                       0.186664
Name: 572, dtype: object
filename         J01_2018.06.27 13_38_19.jpg
probability_x                         0.0005
probability_y                     0.00833996
Name: 573, dtype: object
filename         J01_2018.06.27 13_48_38.jpg
probability_x                         0.9995
probability_y                       0.996005
Name: 574, dtype: object
filename         J01_2018.06.27 13_49_38.jpg
probability_x                         0.0005
probability_y                     0.00266575
Name: 575, dtype: object
filename         J01_2018.06.27 13_50_31.jpg
probability_x                         0.9995
probability_y                          0.998
Name: 576, dtype: object
filename         J01_2018.06.27 13_51_01.jpg
probability_x                      0.0437559
probability_y                      0.0138059
Name: 577, dtype: object
filename       

Name: 623, dtype: object
filename         J01_2018.06.28 14_15_44.jpg
probability_x                       0.391156
probability_y                       0.143051
Name: 624, dtype: object
filename         J01_2018.06.28 14_21_19.jpg
probability_x                         0.9995
probability_y                       0.955801
Name: 625, dtype: object
filename         J01_2018.06.28 14_25_16.jpg
probability_x                         0.9995
probability_y                       0.988429
Name: 626, dtype: object
filename         J01_2018.06.28 14_25_31.jpg
probability_x                         0.0005
probability_y                      0.0295297
Name: 627, dtype: object
filename         J01_2018.06.28 14_30_19.jpg
probability_x                         0.0005
probability_y                       0.393158
Name: 628, dtype: object
filename         J01_2018.06.28 14_32_31.jpg
probability_x                      0.0640352
probability_y                      0.0218839
Name: 629, dtype: object
filename       

In [127]:
for i in range(662):
    if copy3.loc[i,'probability_y']>0.5 and copy3.loc[i,'probability_x']>0.5:
        copy3.loc[i,'probability']= copy3.loc[i,'probability_x']
    elif copy3.loc[i,'probability_y']>0.5 and copy3.loc[i,'probability_x']<0.5:
        copy3.loc[i,'probability']= copy3.loc[i,'probability_y']
    elif copy3.loc[i,'probability_y']<0.5 and copy3.loc[i,'probability_x']<0.5:
        copy3.loc[i,'probability']= copy3.loc[i,'probability_x']
    elif copy3.loc[i,'probability_y']<0.5 and copy3.loc[i,'probability_x']>0.5:
        copy3.loc[i,'probability']= copy3.loc[i,'probability_y']

In [130]:
final = copy3[['filename','probability']]

In [131]:
final.to_csv('submit_mixAUC_LOSS.csv',index=None)

In [20]:
## 加载具有最好验证loss的模型

model.load_weights('InceptionResNetV2_11.40.hdf5')

In [21]:
predictions=model.predict(X_test)

In [22]:
predictions.sum()

229.39594

In [23]:
predictions = predictions.clip(min=0.0005, max=0.9995)

In [24]:
probability=[]
for i in range(len(predictions)):
    probability.append(predictions[i][0])

In [25]:
import os
import pandas as pd
imgs = os.listdir("test/")
df = pd.DataFrame({'filename': imgs,'probability': probability})
df.to_csv('submit_AUC.98.csv',index=None)

# 模型融合

In [3]:
import h5py
with h5py.File('biliao_Data_500') as h:
    X_train = np.array(h['train'])
    X_test = np.array(h['test'])
    y_train = np.array(h['label'])

In [5]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import *
from keras.models import *
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.optimizers import *
from keras.applications import *

Using TensorFlow backend.


In [6]:
def build_MODEL(MODEL,img_size,out_dims,func=None,weights=None,include_top=False):
    inputs = Input((img_size,img_size,3)) #实例化一个tensor
    x = inputs
    x = Lambda(func)(x)
    base_model = MODEL(weights=weights, include_top=include_top) #要提取特征，最后的一层全连接层要去掉
    x = base_model(x)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(out_dims, activation='sigmoid')(x)
    model = Model(inputs, x)
    return model

In [8]:
def FeatureExtract(MODEL,img_size,func=None,weight_path=None):
    base_model = build_MODEL(MODEL,500,out_dims=1,func=func,weights=None)
    base_model.load_weights(weight_path)
    model = Model(inputs=base_model.input, outputs=base_model.layers[-3].output)
    datagen = ImageDataGenerator()
    train_generator = datagen.flow(X_train,batch_size=16, shuffle=False)
    test_generator = datagen.flow(X_test,batch_size=16, shuffle=False)
    train_features = model.predict_generator(train_generator, verbose=1) #得到提取的特征,
    test_features = model.predict_generator(test_generator, verbose=1)
    # 保存bottleneck特征
    with h5py.File('%s_datab.h5'%MODEL.__name__) as h:
        h.create_dataset("train",data = train_features)
        h.create_dataset("test",data = test_features)
        h.create_dataset('label',data = y_train)

In [9]:
from tensorflow.python.keras.applications.inception_resnet_v2 import preprocess_input
FeatureExtract(InceptionResNetV2,500,func=preprocess_input,weight_path='InceptionResNetV2_500_0.97946.hdf5')

41/41 [==============================] - ETA: 13 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 12s 300ms/step


In [10]:
from tensorflow.python.keras.applications.inception_v3 import preprocess_input
FeatureExtract(InceptionV3,500,func=preprocess_input,weight_path='weights_InceptionV3.hdf5')

41/41 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 6s 140ms/step


In [11]:
from tensorflow.python.keras.applications.inception_resnet_v2 import preprocess_input
FeatureExtract(InceptionResNetV2,500,func=preprocess_input,weight_path='weights_InceptionResNetV2.hdf5')

41/41 [==============================] - ETA: 13 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 12s 287ms/step


In [7]:
X_train = []
X_test = []
for filename in ['InceptionV3_datab.h5','InceptionResNetV2_datab.h5','InceptionResNetV2_datab_auc.h5']:
    with h5py.File(filename,'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])
X_train = np.concatenate(X_train,axis=1)
X_test = np.concatenate(X_test,axis=1)

In [8]:
#打乱训练集
from sklearn.utils import shuffle
np.random.seed(2018) #可以设置随机种子，以后每次打乱都是一样的。
X_train,y_train = shuffle(X_train,y_train)

In [11]:
#训练
from keras.layers import *
from keras.models import *
from keras.callbacks import ModelCheckpoint
inputs = Input((X_train.shape[1:]))
x = Dropout(0.5)(inputs)
x = Dense(1, activation='sigmoid')(x)
model = Model(inputs, x)
model.compile(optimizer='adam',
loss='binary_crossentropy',
metrics=[auc_roc])

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=2018)

In [13]:
checkpointer = ModelCheckpoint(filepath='weights.best.loss.b.hdf5',monitor='val_loss',mode='min',
                               verbose=1, save_best_only=True) #保存最好模型权重
reduce = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=5,verbose=1)
adam = Adam(0.00005)
model.compile(optimizer='adam',
loss='binary_crossentropy',
metrics=[auc_roc])
model.load_weights('weights.best.loss.b0.20600.hdf5')
epochs = 100
history = model.fit(X_train, y_train,
validation_data=(X_val, y_val),
epochs=epochs, batch_size=16,callbacks=[checkpointer,reduce],verbose=1)

Train on 1617 samples, validate on 405 samples
Epoch 1/100
1617/1617 [==============================] - ETA: 1:50 - loss: 0.1144 - auc: 0.0000e+0 - ETA: 11s - loss: 0.1342 - auc: 0.8672    - ETA: 3s - loss: 0.1766 - auc: 0.9339 - ETA: 1s - loss: 0.2095 - auc: 0.945 - ETA: 1s - loss: 0.2296 - auc: 0.949 - ETA: 0s - loss: 0.2372 - auc: 0.951 - ETA: 0s - loss: 0.2368 - auc: 0.952 - ETA: 0s - loss: 0.2353 - auc: 0.953 - 2s 988us/step - loss: 0.2354 - auc: 0.9532 - val_loss: 0.1891 - val_auc: 0.9582

Epoch 00001: val_loss improved from inf to 0.18907, saving model to weights.best.loss.b.hdf5
Epoch 2/100
1617/1617 [==============================] - ETA: 0s - loss: 0.1048 - auc: 0.960 - ETA: 0s - loss: 0.1953 - auc: 0.961 - ETA: 0s - loss: 0.2349 - auc: 0.960 - ETA: 0s - loss: 0.2363 - auc: 0.959 - ETA: 0s - loss: 0.2210 - auc: 0.959 - ETA: 0s - loss: 0.2263 - auc: 0.959 - ETA: 0s - loss: 0.2271 - auc: 0.959 - ETA: 0s - loss: 0.2376 - auc: 0.959 - 0s 257us/step - loss: 0.2374 - auc: 0.9596 - 

1617/1617 [==============================] - ETA: 0s - loss: 0.1538 - auc: 0.962 - ETA: 0s - loss: 0.2156 - auc: 0.962 - ETA: 0s - loss: 0.2129 - auc: 0.962 - ETA: 0s - loss: 0.2386 - auc: 0.962 - ETA: 0s - loss: 0.2255 - auc: 0.962 - ETA: 0s - loss: 0.2300 - auc: 0.962 - ETA: 0s - loss: 0.2209 - auc: 0.962 - 0s 256us/step - loss: 0.2224 - auc: 0.9620 - val_loss: 0.1944 - val_auc: 0.9619

Epoch 00017: val_loss did not improve from 0.18686
Epoch 18/100
1617/1617 [==============================] - ETA: 0s - loss: 0.7332 - auc: 0.962 - ETA: 0s - loss: 0.2474 - auc: 0.962 - ETA: 0s - loss: 0.2477 - auc: 0.962 - ETA: 0s - loss: 0.2404 - auc: 0.962 - ETA: 0s - loss: 0.2349 - auc: 0.961 - ETA: 0s - loss: 0.2292 - auc: 0.962 - ETA: 0s - loss: 0.2224 - auc: 0.962 - ETA: 0s - loss: 0.2174 - auc: 0.962 - 0s 263us/step - loss: 0.2176 - auc: 0.9620 - val_loss: 0.1951 - val_auc: 0.9621

Epoch 00018: val_loss did not improve from 0.18686

Epoch 00018: ReduceLROnPlateau reducing learning rate to 1.000


Epoch 00033: val_loss did not improve from 0.18686

Epoch 00033: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-09.
Epoch 34/100
1617/1617 [==============================] - ETA: 0s - loss: 0.0525 - auc: 0.963 - ETA: 0s - loss: 0.1818 - auc: 0.963 - ETA: 0s - loss: 0.2225 - auc: 0.963 - ETA: 0s - loss: 0.2163 - auc: 0.963 - ETA: 0s - loss: 0.2243 - auc: 0.963 - ETA: 0s - loss: 0.2220 - auc: 0.963 - ETA: 0s - loss: 0.2218 - auc: 0.963 - 0s 247us/step - loss: 0.2156 - auc: 0.9630 - val_loss: 0.1952 - val_auc: 0.9630

Epoch 00034: val_loss did not improve from 0.18686
Epoch 35/100
1617/1617 [==============================] - ETA: 0s - loss: 0.1841 - auc: 0.963 - ETA: 0s - loss: 0.1644 - auc: 0.963 - ETA: 0s - loss: 0.1989 - auc: 0.963 - ETA: 0s - loss: 0.2078 - auc: 0.963 - ETA: 0s - loss: 0.2069 - auc: 0.963 - ETA: 0s - loss: 0.2171 - auc: 0.963 - ETA: 0s - loss: 0.2175 - auc: 0.963 - 0s 247us/step - loss: 0.2184 - auc: 0.9630 - val_loss: 0.1952 - val_auc: 0.9630

Epoch 

1617/1617 [==============================] - ETA: 0s - loss: 0.1964 - auc: 0.963 - ETA: 0s - loss: 0.1987 - auc: 0.963 - ETA: 0s - loss: 0.2357 - auc: 0.963 - ETA: 0s - loss: 0.2179 - auc: 0.963 - ETA: 0s - loss: 0.2202 - auc: 0.963 - ETA: 0s - loss: 0.2213 - auc: 0.963 - ETA: 0s - loss: 0.2231 - auc: 0.963 - ETA: 0s - loss: 0.2292 - auc: 0.963 - 0s 262us/step - loss: 0.2265 - auc: 0.9632 - val_loss: 0.1952 - val_auc: 0.9632

Epoch 00050: val_loss did not improve from 0.18686
Epoch 51/100
1617/1617 [==============================] - ETA: 0s - loss: 0.1565 - auc: 0.963 - ETA: 0s - loss: 0.2246 - auc: 0.963 - ETA: 0s - loss: 0.1986 - auc: 0.963 - ETA: 0s - loss: 0.2153 - auc: 0.963 - ETA: 0s - loss: 0.2198 - auc: 0.963 - ETA: 0s - loss: 0.2246 - auc: 0.963 - ETA: 0s - loss: 0.2249 - auc: 0.963 - ETA: 0s - loss: 0.2200 - auc: 0.963 - 0s 261us/step - loss: 0.2199 - auc: 0.9632 - val_loss: 0.1952 - val_auc: 0.9632

Epoch 00051: val_loss did not improve from 0.18686
Epoch 52/100
1617/1617 [=

1617/1617 [==============================] - ETA: 0s - loss: 0.1921 - auc: 0.963 - ETA: 0s - loss: 0.2134 - auc: 0.963 - ETA: 0s - loss: 0.2091 - auc: 0.963 - ETA: 0s - loss: 0.2064 - auc: 0.963 - ETA: 0s - loss: 0.2191 - auc: 0.963 - ETA: 0s - loss: 0.2175 - auc: 0.963 - ETA: 0s - loss: 0.2206 - auc: 0.963 - ETA: 0s - loss: 0.2195 - auc: 0.963 - 0s 266us/step - loss: 0.2175 - auc: 0.9633 - val_loss: 0.1952 - val_auc: 0.9633

Epoch 00066: val_loss did not improve from 0.18686
Epoch 67/100
1617/1617 [==============================] - ETA: 0s - loss: 0.1516 - auc: 0.963 - ETA: 0s - loss: 0.2100 - auc: 0.963 - ETA: 0s - loss: 0.1979 - auc: 0.963 - ETA: 0s - loss: 0.1919 - auc: 0.963 - ETA: 0s - loss: 0.1866 - auc: 0.963 - ETA: 0s - loss: 0.1939 - auc: 0.963 - ETA: 0s - loss: 0.2021 - auc: 0.963 - 0s 251us/step - loss: 0.2145 - auc: 0.9634 - val_loss: 0.1952 - val_auc: 0.9633

Epoch 00067: val_loss did not improve from 0.18686
Epoch 68/100
1617/1617 [==============================] - ETA: 

1617/1617 [==============================] - ETA: 0s - loss: 0.1157 - auc: 0.963 - ETA: 0s - loss: 0.2064 - auc: 0.963 - ETA: 0s - loss: 0.2209 - auc: 0.963 - ETA: 0s - loss: 0.2247 - auc: 0.963 - ETA: 0s - loss: 0.2169 - auc: 0.963 - ETA: 0s - loss: 0.2173 - auc: 0.963 - ETA: 0s - loss: 0.2192 - auc: 0.963 - ETA: 0s - loss: 0.2117 - auc: 0.963 - 0s 275us/step - loss: 0.2133 - auc: 0.9635 - val_loss: 0.1952 - val_auc: 0.9635

Epoch 00082: val_loss did not improve from 0.18686
Epoch 83/100
1617/1617 [==============================] - ETA: 0s - loss: 0.3141 - auc: 0.963 - ETA: 0s - loss: 0.2039 - auc: 0.963 - ETA: 0s - loss: 0.2404 - auc: 0.963 - ETA: 0s - loss: 0.2308 - auc: 0.963 - ETA: 0s - loss: 0.2242 - auc: 0.963 - ETA: 0s - loss: 0.2216 - auc: 0.963 - ETA: 0s - loss: 0.2198 - auc: 0.963 - ETA: 0s - loss: 0.2238 - auc: 0.963 - 0s 289us/step - loss: 0.2220 - auc: 0.9635 - val_loss: 0.1952 - val_auc: 0.9635

Epoch 00083: val_loss did not improve from 0.18686

Epoch 00083: ReduceLROnP

1617/1617 [==============================] - ETA: 0s - loss: 0.5850 - auc: 0.963 - ETA: 0s - loss: 0.2344 - auc: 0.963 - ETA: 0s - loss: 0.2090 - auc: 0.963 - ETA: 0s - loss: 0.2055 - auc: 0.963 - ETA: 0s - loss: 0.2048 - auc: 0.963 - ETA: 0s - loss: 0.2169 - auc: 0.963 - ETA: 0s - loss: 0.2170 - auc: 0.963 - ETA: 0s - loss: 0.2225 - auc: 0.963 - 0s 257us/step - loss: 0.2225 - auc: 0.9635 - val_loss: 0.1952 - val_auc: 0.9635

Epoch 00098: val_loss did not improve from 0.18686

Epoch 00098: ReduceLROnPlateau reducing learning rate to 1.0000000692397185e-22.
Epoch 99/100
1617/1617 [==============================] - ETA: 0s - loss: 0.1263 - auc: 0.963 - ETA: 0s - loss: 0.1879 - auc: 0.963 - ETA: 0s - loss: 0.2092 - auc: 0.963 - ETA: 0s - loss: 0.2130 - auc: 0.963 - ETA: 0s - loss: 0.2237 - auc: 0.963 - ETA: 0s - loss: 0.2251 - auc: 0.963 - ETA: 0s - loss: 0.2182 - auc: 0.963 - 0s 245us/step - loss: 0.2178 - auc: 0.9635 - val_loss: 0.1952 - val_auc: 0.9635

Epoch 00099: val_loss did not im

In [14]:
model.load_weights('weights.best.loss.b.hdf5')

In [15]:
y_pred = model.predict(X_test)
y_pred = y_pred.ravel() #y_pred变为一维
y_pred = y_pred.clip(min=0.002,max=0.998) #限制预测结果在[0.005,0.995]

In [16]:
y_pred.shape

(647,)

In [18]:
import pandas as pd
imgs = os.listdir("xuelang_round1_test_b/")
df = pd.DataFrame({'filename': imgs,'probability': y_pred})
df.to_csv('submit_b_2.csv',index=None)